In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('/Users/champagnepuppy/de-projects/Housing_Price_ML_E2E/data/raw/untouched_raw_original.csv')

In [7]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows

In [8]:
df.shape

(884092, 39)

In [25]:
df.head(2)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
0,2012-03-31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,0.943662,0.142857,0.043478,ATL,30002,2012,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Alpharetta
183322,2012-03-31,156500.0,182188.5,119.789171,140.818182,177.0,0.0,260.0,310.0,74.0,0.966860,0.237288,0.423581,CLT,28205,2012,0.0,0.0,6.0,0.0,17.0,0.0,42.0,2.0,0.0,42087.0,32.9,23739.0,41735.0,20816.0,747.0,163000.0,24348.0,2632.0,39826.0,39826.0,20345.0,137474.419588,Charlotte-Concord-Gastonia


In [10]:
df['date'] = pd.to_datetime(df['date'])
print(df['date'].dt.year.value_counts())

date
2013    74712
2014    74712
2015    74712
2016    74712
2017    74712
2018    74712
2019    74712
2020    74712
2021    74712
2022    74712
2023    74712
2012    62260
Name: count, dtype: int64


##### Avoid Leakage

- train set (2012–2019) is “clean” historical training.

- eval set (2020–2021) is “unseen” during training, but still used for validation and hyperparameter tuning.

- holdout set (2022–2023) is completely untouched until the very end.

In [18]:
# Sort by date (important to avoid shuffling issues)
df = df.sort_values("date")

In [19]:
# Define cutoff dates
cutoff_date_eval = "2020-01-01"   # validation starts
cutoff_date_holdout = "2022-01-01"  # holdout starts

In [20]:
# Train: before 2020
train_df = df[df["date"] < cutoff_date_eval]

# Validation/Eval: 2020–2021
eval_df = df[(df["date"] >= cutoff_date_eval) & (df["date"] < cutoff_date_holdout)]

# Holdout: 2022–2023
holdout_df = df[df["date"] >= cutoff_date_holdout]

print("Train shape:", train_df.shape)
print("Eval shape:", eval_df.shape)
print("Holdout shape:", holdout_df.shape)

Train shape: (585244, 39)
Eval shape: (149424, 39)
Holdout shape: (149424, 39)


In [21]:
train_df.head(2)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
0,2012-03-31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,0.943662,0.142857,0.043478,ATL,30002,2012,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Alpharetta
183322,2012-03-31,156500.0,182188.5,119.789171,140.818182,177.0,0.0,260.0,310.0,74.0,0.966860,0.237288,0.423581,CLT,28205,2012,0.0,0.0,6.0,0.0,17.0,0.0,42.0,2.0,0.0,42087.0,32.9,23739.0,41735.0,20816.0,747.0,163000.0,24348.0,2632.0,39826.0,39826.0,20345.0,137474.419588,Charlotte-Concord-Gastonia


In [22]:
eval_df.head(2)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
272734,2020-01-31,391000.0,399250.0,144.336169,149.398137,298.0,289.0,218.0,100.0,34.0,0.994285,0.265101,0.249135,DEN,80022,2020,6.0,0.0,1.0,0.0,52.0,7.0,22.0,0.0,3.0,51863.0,31.6,25846.0,51472.0,15446.0,1221.0,305900.0,26080.0,1267.0,49042.0,49042.0,23285.0,3.580094e+05,Denver-Aurora-Lakewood
580874,2020-01-31,12112500.0,10725000.0,1571.709470,2947.265773,2.0,2.0,2.0,3.0,196.5,0.915573,0.000000,0.000000,NY,11962,2020,1.0,0.0,0.0,0.0,3.0,0.0,4.0,1.0,0.0,324.0,63.8,84225.0,324.0,952.0,1666.0,2000001.0,111.0,0.0,315.0,315.0,80.0,3.729513e+06,New York-Newark-Jersey City


In [23]:
holdout_df.head(2)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
493994,2022-01-31,590000.0,657450.0,343.870968,301.327241,19.0,10.0,6.0,4.0,48.0,1.010785,0.473684,0.300000,NY,7702,2022,14.0,0.0,1.0,2.0,29.0,41.0,42.0,3.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.934719e+05,New York-Newark-Jersey City
835362,2022-01-31,1400000.0,1522500.0,857.142857,891.865994,33.0,34.0,28.0,11.0,28.0,1.028703,0.454545,0.441176,SF,94965,2022,20.0,4.0,0.0,1.0,39.0,66.0,20.0,0.0,11.0,11821.0,51.4,97381.0,11760.0,6463.0,2443.0,1173700.0,6859.0,371.0,11565.0,11565.0,4613.0,1.582472e+06,San Francisco-Oakland-Berkeley


In [24]:
# Save splits
train_df.to_csv("/Users/champagnepuppy/de-projects/Housing_Price_ML_E2E/data/raw/train.csv", index=False)
eval_df.to_csv("/Users/champagnepuppy/de-projects/Housing_Price_ML_E2E/data/raw/eval.csv", index=False)
holdout_df.to_csv("/Users/champagnepuppy/de-projects/Housing_Price_ML_E2E/data/raw/holdout.csv", index=False)